In [1]:
import pandas as pd
import numpy as np

In [2]:
purchases = pd.read_csv(r'C:\datasets\retailhero-uplift\data\purchases.csv')
uplift_train = pd.read_csv(r'C:\datasets\retailhero-uplift\data\uplift_train.csv')
uplift_test = pd.read_csv(r'C:\datasets\retailhero-uplift\data\uplift_test.csv')
uplift_sample_submission = pd.read_csv(r'C:\datasets\retailhero-uplift\data\uplift_sample_submission.csv')

In [3]:
products = pd.read_csv(r'C:\datasets\retailhero-uplift\data\products.csv')

In [4]:
clients = pd.read_csv(r'C:\datasets\retailhero-uplift\data\clients.csv')

*Перобразуем форматы времени в числа, чтобы было проще работать дальше*

In [5]:
clients['first_issue_date'] =  (pd.to_datetime(clients['first_issue_date']).astype("int64")) // 10 ** 9
clients['first_redeem_date'] =  (pd.to_datetime(clients['first_redeem_date']).astype("int64")) // 10 ** 9
clients['timedelta'] = clients['first_redeem_date'] - clients['first_issue_date']
clients.loc[clients['first_redeem_date']<0, 'first_redeem_date'] = 0
clients.loc[clients['timedelta']<0, 'timedelta'] = 0

In [6]:
clients['gender'] = pd.get_dummies(clients['gender'])

In [7]:
clients.head()

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta
0,000012768d,1501947648,1515094207,45,0,13146559
1,000036f903,1491832463,1492951076,72,1,1118613
2,000048b7a6,1544880791,0,68,1,0
3,000073194a,1495544174,1511522281,60,1,15978107
4,00007c7133,1495469828,1546276653,67,0,50806825


***
***

*Поработаем с purchases*

In [8]:
purchases.head()

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN


In [9]:
test = purchases.pivot_table(index=['client_id', 'transaction_id']).reset_index()

In [10]:
test.head()

,client_id,transaction_id,express_points_received,express_points_spent,product_quantity,purchase_sum,regular_points_received,regular_points_spent,trn_sum_from_iss,trn_sum_from_red
0,000012768d,6a0e96d0bc,0.0,0.0,0.812500,803.0,8.0,0.0,50.250000,NaN
1,000012768d,7e3e2e3984,0.0,0.0,1.105263,1007.0,10.0,0.0,53.000000,NaN
2,000012768d,b34f23306e,0.0,0.0,1.000000,419.0,2.0,0.0,69.833333,NaN
3,000012768d,c1ca85d462,0.0,0.0,1.272727,574.0,5.7,0.0,52.272727,NaN
4,000036f903,0a3d640bf4,0.0,0.0,1.000000,700.0,7.0,0.0,87.500000,NaN


In [11]:
test1 = test.groupby('client_id').agg({'transaction_id': 'count', 
                                       'purchase_sum': 'mean', 
                                       'regular_points_received': 'sum', 
                                       'express_points_received': 'sum', 
                                       'regular_points_spent': 'sum', 
                                       'express_points_spent': 'sum'}).reset_index()

In [12]:
test1.head()

,client_id,transaction_id,purchase_sum,regular_points_received,express_points_received,regular_points_spent,express_points_spent
0,000012768d,4,700.750000,25.7,0.0,0.0,0.0
1,000036f903,32,306.406250,54.9,60.0,0.0,0.0
2,000048b7a6,8,471.500000,26.5,0.0,0.0,0.0
3,000073194a,17,564.788235,74.9,0.0,-96.0,0.0
4,00007c7133,11,610.894545,56.6,0.0,-240.0,0.0


In [13]:
test1['points_received'] = test1['regular_points_received'] + test1['express_points_received']
test1['points_spent'] = abs(test1['regular_points_spent'] + test1['express_points_spent'])

In [14]:
test1 = test1.drop(['regular_points_received', 'express_points_received', 'regular_points_spent', 'express_points_spent'], axis=1)

In [15]:
test1.columns = ['client_id', 'transaction_count', 'purchase_trans_mean', 'points_received', 'points_spent']

In [16]:
test2 = test.groupby('client_id').agg({'purchase_sum': 'sum'}).reset_index()
test2.columns = ['client_id', 'purchase_trans_sum']

In [21]:
test2

,client_id,purchase_trans_sum
0,000012768d,2803.00
1,000036f903,9805.00
2,000048b7a6,3772.00
3,000073194a,9601.40
4,00007c7133,6719.84
...,...,...
400157,fffece623e,6530.01
400158,ffff3dfff8,7264.84
400159,ffffaab9da,3791.00
400160,ffffeb5619,15226.22


In [18]:
clients = clients.merge(test1)
clients = clients.merge(test2)

In [22]:
clients

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta,transaction_count,purchase_trans_mean,points_received,points_spent,purchase_trans_sum
0,000012768d,1501947648,1515094207,45,0,13146559,4,700.750000,25.7,0.0,2803.00
1,000036f903,1491832463,1492951076,72,1,1118613,32,306.406250,114.9,0.0,9805.00
2,000048b7a6,1544880791,0,68,1,0,8,471.500000,26.5,0.0,3772.00
3,000073194a,1495544174,1511522281,60,1,15978107,17,564.788235,74.9,96.0,9601.40
4,00007c7133,1495469828,1546276653,67,0,50806825,11,610.894545,56.6,240.0,6719.84
...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,1526224170,1545937599,67,0,19713429,24,272.083750,38.4,68.0,6530.01
400158,ffff3dfff8,1541088412,1544204027,56,1,3115615,17,427.343529,117.9,182.0,7264.84
400159,ffffaab9da,1503686604,1512755599,23,1,9068995,7,541.571429,34.0,6.0,3791.00
400160,ffffeb5619,1512480388,0,62,0,0,7,2175.174286,117.5,0.0,15226.22


In [23]:
clients['percent_points'] = (clients['points_received'] / clients['purchase_trans_sum']) * 100

In [24]:
clients

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta,transaction_count,purchase_trans_mean,points_received,points_spent,purchase_trans_sum,percent_points
0,000012768d,1501947648,1515094207,45,0,13146559,4,700.750000,25.7,0.0,2803.00,0.916875
1,000036f903,1491832463,1492951076,72,1,1118613,32,306.406250,114.9,0.0,9805.00,1.171851
2,000048b7a6,1544880791,0,68,1,0,8,471.500000,26.5,0.0,3772.00,0.702545
3,000073194a,1495544174,1511522281,60,1,15978107,17,564.788235,74.9,96.0,9601.40,0.780095
4,00007c7133,1495469828,1546276653,67,0,50806825,11,610.894545,56.6,240.0,6719.84,0.842282
...,...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,1526224170,1545937599,67,0,19713429,24,272.083750,38.4,68.0,6530.01,0.588054
400158,ffff3dfff8,1541088412,1544204027,56,1,3115615,17,427.343529,117.9,182.0,7264.84,1.622885
400159,ffffaab9da,1503686604,1512755599,23,1,9068995,7,541.571429,34.0,6.0,3791.00,0.896861
400160,ffffeb5619,1512480388,0,62,0,0,7,2175.174286,117.5,0.0,15226.22,0.771695


In [26]:
test_store = purchases.pivot_table(index=['client_id', 'store_id'], values='purchase_sum', aggfunc='sum').reset_index()

In [27]:
test_store

,client_id,store_id,purchase_sum
0,000012768d,017c89b915,15362.00
1,000012768d,54a4a11a29,19133.00
2,000012768d,ed8d2683fd,6314.00
3,000036f903,116ef2dfce,1862.00
4,000036f903,3c6c3b17f2,12556.00
...,...,...,...
1169839,ffffeb5619,7c1d114de1,27250.90
1169840,ffffeb5619,b77dc8c2a3,252234.78
1169841,fffff6ce77,5ec64bdccb,43654.00
1169842,fffff6ce77,6e4faad02e,130543.31


In [28]:
test_store_mean = test_store.groupby('client_id').agg({'purchase_sum':'mean'}).reset_index()
test_store_mean.columns = ['client_id', 'purchase_store_mean']

In [29]:
test_store_sum = test_store.groupby('client_id').agg({'purchase_sum':'sum'}).reset_index()
test_store_sum.columns = ['client_id', 'purchase_store_sum']

In [30]:
test_store_agg = test_store_sum.merge(test_store_mean)

In [31]:
clients = clients.merge(test_store_agg)

In [32]:
clients

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta,transaction_count,purchase_trans_mean,points_received,points_spent,purchase_trans_sum,percent_points,purchase_store_sum,purchase_store_mean
0,000012768d,1501947648,1515094207,45,0,13146559,4,700.750000,25.7,0.0,2803.00,0.916875,40809.00,13603.000000
1,000036f903,1491832463,1492951076,72,1,1118613,32,306.406250,114.9,0.0,9805.00,1.171851,58765.00,11753.000000
2,000048b7a6,1544880791,0,68,1,0,8,471.500000,26.5,0.0,3772.00,0.702545,29724.00,14862.000000
3,000073194a,1495544174,1511522281,60,1,15978107,17,564.788235,74.9,96.0,9601.40,0.780095,62719.92,62719.920000
4,00007c7133,1495469828,1546276653,67,0,50806825,11,610.894545,56.6,240.0,6719.84,0.842282,53998.72,26999.360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,1526224170,1545937599,67,0,19713429,24,272.083750,38.4,68.0,6530.01,0.588054,43103.46,14367.820000
400158,ffff3dfff8,1541088412,1544204027,56,1,3115615,17,427.343529,117.9,182.0,7264.84,1.622885,65810.71,65810.710000
400159,ffffaab9da,1503686604,1512755599,23,1,9068995,7,541.571429,34.0,6.0,3791.00,0.896861,41092.00,41092.000000
400160,ffffeb5619,1512480388,0,62,0,0,7,2175.174286,117.5,0.0,15226.22,0.771695,279485.68,139742.840000


In [33]:
test_store_prod = purchases.pivot_table(index=['client_id', 'store_id'], values='product_id', aggfunc='count').reset_index()

In [34]:
test_store_prod_sum_prod_store = test_store_prod.groupby('client_id').agg({'product_id': 'sum'}).reset_index()
test_store_prod_sum_prod_store.columns = ['client_id', 'sum_prod_store']
test_store_prod_mean_prod_store = test_store_prod.groupby('client_id').agg({'product_id': 'mean'}).reset_index()
test_store_prod_mean_prod_store.columns = ['client_id', 'mean_prod_store']
test_store_prod_sum_prod_store = test_store_prod_sum_prod_store.merge(test_store_prod_mean_prod_store)

In [35]:
test_store_prod_sum_prod_store

,client_id,sum_prod_store,mean_prod_store
0,000012768d,52,17.333333
1,000036f903,162,32.400000
2,000048b7a6,56,28.000000
3,000073194a,82,82.000000
4,00007c7133,83,41.500000
...,...,...,...
400157,fffece623e,117,39.000000
400158,ffff3dfff8,96,96.000000
400159,ffffaab9da,60,60.000000
400160,ffffeb5619,99,49.500000


In [36]:
test_store1 = test_store.groupby('client_id').agg({'store_id': 'count'}).reset_index()
test_store1.columns = ['client_id', 'store_count']

In [37]:
test_store_prod_sum_prod_store = test_store_prod_sum_prod_store.merge(test_store1)

In [38]:
test_store_prod_sum_prod_store

,client_id,sum_prod_store,mean_prod_store,store_count
0,000012768d,52,17.333333,3
1,000036f903,162,32.400000,5
2,000048b7a6,56,28.000000,2
3,000073194a,82,82.000000,1
4,00007c7133,83,41.500000,2
...,...,...,...,...
400157,fffece623e,117,39.000000,3
400158,ffff3dfff8,96,96.000000,1
400159,ffffaab9da,60,60.000000,1
400160,ffffeb5619,99,49.500000,2


In [39]:
clients = clients.merge(test_store_prod_sum_prod_store)

In [40]:
clients

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta,transaction_count,purchase_trans_mean,points_received,points_spent,purchase_trans_sum,percent_points,purchase_store_sum,purchase_store_mean,sum_prod_store,mean_prod_store,store_count
0,000012768d,1501947648,1515094207,45,0,13146559,4,700.750000,25.7,0.0,2803.00,0.916875,40809.00,13603.000000,52,17.333333,3
1,000036f903,1491832463,1492951076,72,1,1118613,32,306.406250,114.9,0.0,9805.00,1.171851,58765.00,11753.000000,162,32.400000,5
2,000048b7a6,1544880791,0,68,1,0,8,471.500000,26.5,0.0,3772.00,0.702545,29724.00,14862.000000,56,28.000000,2
3,000073194a,1495544174,1511522281,60,1,15978107,17,564.788235,74.9,96.0,9601.40,0.780095,62719.92,62719.920000,82,82.000000,1
4,00007c7133,1495469828,1546276653,67,0,50806825,11,610.894545,56.6,240.0,6719.84,0.842282,53998.72,26999.360000,83,41.500000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,1526224170,1545937599,67,0,19713429,24,272.083750,38.4,68.0,6530.01,0.588054,43103.46,14367.820000,117,39.000000,3
400158,ffff3dfff8,1541088412,1544204027,56,1,3115615,17,427.343529,117.9,182.0,7264.84,1.622885,65810.71,65810.710000,96,96.000000,1
400159,ffffaab9da,1503686604,1512755599,23,1,9068995,7,541.571429,34.0,6.0,3791.00,0.896861,41092.00,41092.000000,60,60.000000,1
400160,ffffeb5619,1512480388,0,62,0,0,7,2175.174286,117.5,0.0,15226.22,0.771695,279485.68,139742.840000,99,49.500000,2


*Вытащим сколько разных типов товаров покупает клиент*

In [41]:
clients = clients.merge(purchases.pivot_table(index=['client_id'], 
                                       values='product_id', 
                                       aggfunc=pd.Series.nunique).reset_index())

In [42]:
clients

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta,transaction_count,purchase_trans_mean,points_received,points_spent,purchase_trans_sum,percent_points,purchase_store_sum,purchase_store_mean,sum_prod_store,mean_prod_store,store_count,product_id
0,000012768d,1501947648,1515094207,45,0,13146559,4,700.750000,25.7,0.0,2803.00,0.916875,40809.00,13603.000000,52,17.333333,3,46
1,000036f903,1491832463,1492951076,72,1,1118613,32,306.406250,114.9,0.0,9805.00,1.171851,58765.00,11753.000000,162,32.400000,5,96
2,000048b7a6,1544880791,0,68,1,0,8,471.500000,26.5,0.0,3772.00,0.702545,29724.00,14862.000000,56,28.000000,2,44
3,000073194a,1495544174,1511522281,60,1,15978107,17,564.788235,74.9,96.0,9601.40,0.780095,62719.92,62719.920000,82,82.000000,1,68
4,00007c7133,1495469828,1546276653,67,0,50806825,11,610.894545,56.6,240.0,6719.84,0.842282,53998.72,26999.360000,83,41.500000,2,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,1526224170,1545937599,67,0,19713429,24,272.083750,38.4,68.0,6530.01,0.588054,43103.46,14367.820000,117,39.000000,3,83
400158,ffff3dfff8,1541088412,1544204027,56,1,3115615,17,427.343529,117.9,182.0,7264.84,1.622885,65810.71,65810.710000,96,96.000000,1,86
400159,ffffaab9da,1503686604,1512755599,23,1,9068995,7,541.571429,34.0,6.0,3791.00,0.896861,41092.00,41092.000000,60,60.000000,1,53
400160,ffffeb5619,1512480388,0,62,0,0,7,2175.174286,117.5,0.0,15226.22,0.771695,279485.68,139742.840000,99,49.500000,2,75


In [43]:
test_trans_prod = purchases.pivot_table(index=['client_id', 'transaction_id'], 
                                        values='product_id', 
                                        aggfunc='count').reset_index()
test_trans_prod_mean = test_trans_prod.groupby('client_id').agg({'product_id': 'mean'}).reset_index()
test_trans_prod_mean.columns = ['client_id', 'mean_prod_trans']

In [44]:
clients = clients.merge(test_trans_prod_mean)

In [45]:
clients

,client_id,first_issue_date,first_redeem_date,age,gender,timedelta,transaction_count,purchase_trans_mean,points_received,points_spent,purchase_trans_sum,percent_points,purchase_store_sum,purchase_store_mean,sum_prod_store,mean_prod_store,store_count,product_id,mean_prod_trans
0,000012768d,1501947648,1515094207,45,0,13146559,4,700.750000,25.7,0.0,2803.00,0.916875,40809.00,13603.000000,52,17.333333,3,46,13.000000
1,000036f903,1491832463,1492951076,72,1,1118613,32,306.406250,114.9,0.0,9805.00,1.171851,58765.00,11753.000000,162,32.400000,5,96,5.062500
2,000048b7a6,1544880791,0,68,1,0,8,471.500000,26.5,0.0,3772.00,0.702545,29724.00,14862.000000,56,28.000000,2,44,7.000000
3,000073194a,1495544174,1511522281,60,1,15978107,17,564.788235,74.9,96.0,9601.40,0.780095,62719.92,62719.920000,82,82.000000,1,68,4.823529
4,00007c7133,1495469828,1546276653,67,0,50806825,11,610.894545,56.6,240.0,6719.84,0.842282,53998.72,26999.360000,83,41.500000,2,64,7.545455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400157,fffece623e,1526224170,1545937599,67,0,19713429,24,272.083750,38.4,68.0,6530.01,0.588054,43103.46,14367.820000,117,39.000000,3,83,4.875000
400158,ffff3dfff8,1541088412,1544204027,56,1,3115615,17,427.343529,117.9,182.0,7264.84,1.622885,65810.71,65810.710000,96,96.000000,1,86,5.647059
400159,ffffaab9da,1503686604,1512755599,23,1,9068995,7,541.571429,34.0,6.0,3791.00,0.896861,41092.00,41092.000000,60,60.000000,1,53,8.571429
400160,ffffeb5619,1512480388,0,62,0,0,7,2175.174286,117.5,0.0,15226.22,0.771695,279485.68,139742.840000,99,49.500000,2,75,14.142857


***

 *Посмотрим на таблицу product*

In [46]:
products.head()

,product_id,level_1,level_2,level_3,level_4,segment_id,brand_id,vendor_id,netto,is_own_trademark,is_alcohol
0,0003020d3c,c3d3a8e8c6,c2a3ea8d5e,b7cda0ec0c,6376f2a852,123.0,394a54a7c1,9eaff48661,0.400,0,0
1,0003870676,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,acd3dd483f,10486c3cf0,0.680,0,0
2,0003ceaf69,c3d3a8e8c6,f2333c90fb,419bc5b424,f6148afbc0,271.0,f597581079,764e660dda,0.500,0,0
3,000701e093,ec62ce61e3,4202626fcb,88a515c084,48cf3d488f,172.0,54a90fe769,03c2d70bad,0.112,0,0
4,0007149564,e344ab2e71,52f13dac0c,d3cfe81323,6dc544533f,105.0,63417fe1f3,f329130198,0.600,0,0


*Вытащим признаки своей торговой марки и алкоголя*

In [47]:
clients.to_csv(r'C:\datasets\retailhero-uplift\data\final_clients2.csv')